In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df_infos = pd.read_csv('../saved/infos.csv')
fDataReNorms = np.load('../saved/fDataReNorms.npy')

In [4]:
df_infos

,Unnamed: 0,ptID,Sex,Age,AF,I-AVB,LBBB,Normal,PAC,PVC,RBBB,STD,STE,mean_intervals,std_intervals,interval_len
0,0,A0001,0,74,False,False,False,False,False,False,True,False,False,299.055556,32.777825,303
1,1,A0001,0,74,False,False,False,False,False,False,True,False,False,299.055556,32.777825,292
2,2,A0001,0,74,False,False,False,False,False,False,True,False,False,299.055556,32.777825,296
3,3,A0001,0,74,False,False,False,False,False,False,True,False,False,299.055556,32.777825,296
4,4,A0001,0,74,False,False,False,False,False,False,True,False,False,299.055556,32.777825,298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104423,104423,A6813,0,60,False,False,True,False,False,False,False,False,False,511.076923,20.409129,529
104424,104424,A6813,0,60,False,False,True,False,False,False,False,False,False,511.076923,20.409129,526
104425,104425,A6813,0,60,False,False,True,False,False,False,False,False,False,511.076923,20.409129,516
104426,104426,A6813,0,60,False,False,True,False,False,False,False,False,False,511.076923,20.409129,488


In [6]:
# fDataReNormAFs = fDataReNorms[df_infos['AF']]

# for fDataReNormAF in fDataReNormAFs[:10]:
#     plt.plot(fDataReNormAF[0])
#     plt.show()

In [7]:
from collections import Counter
from global_vars import headers, labels
print(Counter(np.sum(df_infos[labels], axis=1)))

Counter({1: 96211, 2: 8079, 3: 138})


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [9]:
y = torch.FloatTensor(df_infos[labels].values).to(device)

In [10]:
X = torch.FloatTensor(fDataReNorms).to(device)

In [11]:
class SimpleNet(nn.Module):
    def __init__(self, verbose=False):
        super(SimpleNet, self).__init__()
        self.fc1        = nn.Linear(250, 250)
        self.fc2        = nn.Linear(250, 10)
        self.fc3        = nn.Linear(120, 10)
        self.fc4        = nn.Linear(10, 9)
        self.dropout = nn.Dropout(p=0.5)
        self.verbose=verbose

    def forward(self, x):
        if self.verbose:
            print('0: ', x.shape)
        x = self.fc1( x )     
        x = self.dropout(x)
        if self.verbose:
            print('1: ', x.shape)
        x = F.relu(   x )     
        x = self.fc2( x )    
        if self.verbose:
            print('2: ', x.shape)
        x = F.relu(   x )      
        if self.verbose:
            print('3: ', x.shape)
        x = x.view(x.shape[0], -1)
        x = self.fc3( x )  
        if self.verbose:
            print('4: ', x.shape)
        x = F.relu(   x )   
        x = self.fc4(x)
        if self.verbose:
            print('5: ', x.shape)
        return x

model = SimpleNet(verbose=True)
model.forward(X.cpu())

0:  torch.Size([104428, 12, 250])
1:  torch.Size([104428, 12, 250])
2:  torch.Size([104428, 12, 10])
3:  torch.Size([104428, 12, 10])
4:  torch.Size([104428, 10])
5:  torch.Size([104428, 9])


tensor([[0.0032, 0.1503, 0.2095,  ..., 0.2048, 0.0963, 0.2829],
        [0.0022, 0.1476, 0.2175,  ..., 0.2116, 0.0957, 0.2843],
        [0.0046, 0.1509, 0.2108,  ..., 0.2063, 0.0970, 0.2822],
        ...,
        [0.0099, 0.1520, 0.2104,  ..., 0.2078, 0.0926, 0.2830],
        [0.0016, 0.1488, 0.2129,  ..., 0.2074, 0.0963, 0.2838],
        [0.0014, 0.1537, 0.2083,  ..., 0.2014, 0.1040, 0.2785]],
       grad_fn=<AddmmBackward>)

In [12]:
# from torch.utils.data import TensorDataset, DataLoader, WeightedRandomSampler
# all_dataset = TensorDataset(X, y)

In [ ]:
#%%
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc
from evaluate_12ECG_score import compute_beta_score, compute_auc

# def evaluate_beta(output, y):
    
#     accuracy,f_measure,f_beta,g_beta = compute_beta_score(labels=y, 
#                        output=output, 
#                        beta=2, num_classes=1)
    
#     auroc, auprc = compute_auc(labels=y, 
#                                 probabilities=output,
#                                 num_classes=1)

#     return accuracy,f_measure,f_beta,g_beta, auroc, auprc


def confusion(prediction, truth):
    
    """ Returns the confusion matrix for the values in the `prediction` and `truth`
    tensors, i.e. the amount of positions where the values of `prediction`
    and `truth` are
    - 1 and 1 (True Positive)
    - 1 and 0 (False Positive)
    - 0 and 0 (True Negative)
    - 0 and 1 (False Negative)
    https://gist.github.com/the-bass/cae9f3976866776dea17a5049013258d
    """

    confusion_vector = prediction / truth
    # Element-wise division of the 2 tensors returns a new tensor which holds a
    # unique value for each case:
    #   1     where prediction and truth are 1 (True Positive)
    #   inf   where prediction is 1 and truth is 0 (False Positive)
    #   nan   where prediction and truth are 0 (True Negative)
    #   0     where prediction is 0 and truth is 1 (False Negative)

    true_positives = torch.sum(confusion_vector == 1).item()
    false_positives = torch.sum(confusion_vector == float('inf')).item()
    true_negatives = torch.sum(torch.isnan(confusion_vector)).item()
    false_negatives = torch.sum(confusion_vector == 0).item()

    return true_positives, false_positives, true_negatives, false_negatives

def binary_acc(y_preds, y_tests, beta=2):
    accs = []
    fmeasures = []
    fbetas = []
    gbetas = []
    aurocs = []
    auprcs = []
    for i in range(9):
        y_pred, y_test = y_preds[:,i], y_tests[:,i]
        
        # prob
        y_pred_prob = torch.sigmoid(y_pred)
        
        # auroc
        y_test_numpy = y_test.data.cpu().numpy()
        y_pred_prob_numpy = y_pred_prob.data.cpu().numpy()
        auroc = roc_auc_score(y_test_numpy, y_pred_prob_numpy)
        
        # auprc
        precision, recall, thresholds = precision_recall_curve(y_test_numpy, y_pred_prob_numpy)
        auprc = auc(recall, precision)

        # binary result
        y_pred_tag = torch.round(y_pred_prob)
        tp, fp, tn, fn = confusion(y_pred_tag, y_test)
        
        # acc, fmeasure, fbeta, gbeta
        acc = float(tp + tn) / float(tp + fp + fn + tn)
        fmeasure = float(2 * tp) / float(2 * tp + fp + fn)
        fbeta = float((1+beta**2)* tp) / float(((1+beta**2)*tp) + (fn*beta**2) + fp)
        gbeta = float(tp) / float(tp + fp + beta*fn)
        
        # old way to cal acc:
        #correct_results_sum = (y_pred_tag == y_test).sum().float()
        #acc = true_positives/y_test.shape[0]
        #acc = torch.round(acc * 100)

        accs.append(acc)#.data.cpu().numpy())
        fbetas.append(fbeta)#.data.cpu().numpy())
        fmeasures.append(fmeasure)
        gbetas.append(gbeta)
        aurocs.append(auroc)
        auprcs.append(auprc)
    return np.mean(accs), np.mean(fbetas), np.mean(fmeasures), np.mean(gbetas), np.mean(aurocs), np.mean(auprcs)

from snippets.pytorchtools import EarlyStopping
from sklearn.model_selection import KFold
import time

st = time.time()
patience = 200
kf = KFold(10)
batch_size=20000

saved_dir = '../saved/modelTrainNN/'
weights = 1.0/np.sum(y.cpu().numpy(),axis=0)
weights /= np.sum(weights)
class_weights = torch.Tensor(weights).to(device)
sampler = torch.utils.data.sampler.WeightedRandomSampler(class_weights, batch_size)

for i, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    
    if i == 0:
        continue
    
#     trainDataset = TensorDataset(X[train_idx], y[train_idx])
#     testDataset = TensorDataset(X[test_idx], y[test_idx])
    
#     trainLoader = torch.utils.data.DataLoader(trainDataset, batch_size = batch_size, sampler = sampler)
#     testLoader = torch.utils.data.DataLoader(testDataset, batch_size = batch_size, shuffle = False)

    model = SimpleNet().to(device)
    learning_rate = 0.01
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    criterion = nn.BCEWithLogitsLoss()#pos_weight=class_weights)

    losses_train = []
    losses_test = []

    avg_losses_train = []
    avg_losses_test = []


    early_stopping = EarlyStopping(patience, verbose=False, 
                                  saved_dir=saved_dir, 
                                   save_name='SimpleNet'+str(i))
    epoch = 0
    auroc = 0
    auprc = 0
    accuracy = 0
    fmeasure = 0
    fbeta = 0
    gbeta = 0
    for epoch in range(100000):
        optimizer.zero_grad()
        model.train()
#         for X_train, y_train in trainLoader:
#             output_train = model(X_train)
#             loss_train = criterion(output_train, y_train)
#             losses_train.append(loss_train.item())
#             loss_train.backward()
#             optimizer.step()

        output_train = model(X[train_idx])
        loss_train = criterion(output_train, y[train_idx])
        loss_train.backward()
        optimizer.step()
        losses_train.append(loss_train.item())
        avg_loss_train = np.average(losses_train)
        avg_losses_train.append(avg_loss_train)
        
        
        with torch.no_grad():
            model.eval()
            
#             for X_test, y_test in testLoader:        
#                 output_test = model(X_test)
#                 loss_test = criterion(output_test, y_test)
#                 losses_test.append(loss_test.item())

            output_test = model(X[test_idx])
            loss_test = criterion(output_test, y[test_idx])
            losses_test.append(loss_test.item())
            avg_loss_test = np.average(losses_test)
            avg_losses_test.append(avg_loss_test)

        if epoch % 10 == 0:
#             output_train = model(X[train_idx])
            acc, fmeasure, fbeta, gbeta, auroc, auprc = binary_acc(output_train, y[train_idx])
            
#             output_test = model(X[test_idx])
            acc2, fmeasure2, fbeta2, gbeta2, auroc2, auprc2 = binary_acc(output_test, y[test_idx])
            #accuracy_p = ' '.join(['{:.2f},'.format(acc)for acc in accuracy])
            #accuracy2_p = ' '.join(['{:.2f},'.format(acc)for acc in accuracy2])
            print('S{}/{} {:.2f} min|\nTrain Loss: {:.6f}, Acc: {:.2f}, F: {:.2f}, Fbeta: {:.2f}, gbeta: {:.2f}, auroc: {:.2f}, auprc: {:.2f} |\nValid Loss: {:.6f}, Acc: {:.2f}, F: {:.2f}, Fbeta: {:.2f}, gbeta: {:.2f}, auroc: {:.2f}, auprc: {:.2f}\n '.format(
                i, epoch, (time.time()-st)/60,
                avg_loss_train, acc, fmeasure, fbeta, gbeta, auroc, auprc, 
                avg_loss_test, acc2, fmeasure2, fbeta2, gbeta2, auroc2, auprc2))

        early_stopping(avg_loss_test, model)

        if early_stopping.early_stop:
            print("Early stopping")
            break
            
    output_string = 'AUROC|AUPRC|Accuracy|F-measure|Fbeta-measure|Gbeta-measure\n{:.3f}|{:.3f}|{:.3f}|{:.3f}|{:.3f}|{:.3f}'.format(auroc2,auprc2,acc2,fmeasure2,fbeta2,gbeta)
    print(output_string)     
    with open(saved_dir+'score'+ str(i)+ '_epoch' + str(epoch) + '.txt', 'w') as f:
        f.write(output_string)

    avg_losses_train = np.array(avg_losses_train)
    avg_losses_test = np.array(avg_losses_test)
    
    np.save(saved_dir + 'avg_losses_train' + str(i) + '_epoch' + str(epoch), avg_losses_train)
    np.save(saved_dir + 'avg_losses_test' + str(i) + '_epoch' + str(epoch), avg_losses_test)
    

S1/0 0.02 min|
Train Loss: 0.689482, Acc: 0.44, F: 0.19, Fbeta: 0.10, gbeta: 0.05, auroc: 0.50, auprc: 0.11 |
Valid Loss: 0.677577, Acc: 0.54, F: 0.13, Fbeta: 0.07, gbeta: 0.04, auroc: 0.60, auprc: 0.17
 
S1/10 0.09 min|
Train Loss: 0.558307, Acc: 0.80, F: 0.05, Fbeta: 0.03, gbeta: 0.01, auroc: 0.60, auprc: 0.19 |
Valid Loss: 0.541737, Acc: 0.79, F: 0.04, Fbeta: 0.02, gbeta: 0.01, auroc: 0.60, auprc: 0.20
 
S1/20 0.17 min|
Train Loss: 0.468968, Acc: 0.89, F: 0.07, Fbeta: 0.08, gbeta: 0.04, auroc: 0.59, auprc: 0.20 |
Valid Loss: 0.457949, Acc: 0.89, F: 0.07, Fbeta: 0.08, gbeta: 0.05, auroc: 0.58, auprc: 0.21
 
S1/30 0.24 min|
Train Loss: 0.421248, Acc: 0.90, F: 0.08, Fbeta: 0.09, gbeta: 0.05, auroc: 0.64, auprc: 0.21 |
Valid Loss: 0.414797, Acc: 0.89, F: 0.08, Fbeta: 0.09, gbeta: 0.05, auroc: 0.63, auprc: 0.22
 
S1/40 0.32 min|
Train Loss: 0.393398, Acc: 0.90, F: 0.08, Fbeta: 0.09, gbeta: 0.06, auroc: 0.70, auprc: 0.27 |
Valid Loss: 0.390122, Acc: 0.89, F: 0.08, Fbeta: 0.09, gbeta: 0.06

S1/400 3.01 min|
Train Loss: 0.240206, Acc: 0.93, F: 0.48, Fbeta: 0.51, gbeta: 0.32, auroc: 0.89, auprc: 0.64 |
Valid Loss: 0.252487, Acc: 0.92, F: 0.46, Fbeta: 0.50, gbeta: 0.29, auroc: 0.88, auprc: 0.61
 
S1/410 3.08 min|
Train Loss: 0.239002, Acc: 0.93, F: 0.50, Fbeta: 0.54, gbeta: 0.34, auroc: 0.89, auprc: 0.65 |
Valid Loss: 0.251545, Acc: 0.92, F: 0.46, Fbeta: 0.50, gbeta: 0.29, auroc: 0.88, auprc: 0.62
 
S1/420 3.16 min|
Train Loss: 0.237822, Acc: 0.93, F: 0.50, Fbeta: 0.54, gbeta: 0.33, auroc: 0.89, auprc: 0.65 |
Valid Loss: 0.250630, Acc: 0.92, F: 0.47, Fbeta: 0.51, gbeta: 0.30, auroc: 0.88, auprc: 0.62
 
S1/430 3.23 min|
Train Loss: 0.236672, Acc: 0.93, F: 0.50, Fbeta: 0.54, gbeta: 0.34, auroc: 0.90, auprc: 0.65 |
Valid Loss: 0.249753, Acc: 0.92, F: 0.47, Fbeta: 0.51, gbeta: 0.30, auroc: 0.88, auprc: 0.62
 
S1/440 3.31 min|
Train Loss: 0.235568, Acc: 0.93, F: 0.50, Fbeta: 0.54, gbeta: 0.34, auroc: 0.90, auprc: 0.65 |
Valid Loss: 0.248924, Acc: 0.92, F: 0.47, Fbeta: 0.51, gbeta

S1/800 6.42 min|
Train Loss: 0.211020, Acc: 0.93, F: 0.54, Fbeta: 0.58, gbeta: 0.37, auroc: 0.91, auprc: 0.70 |
Valid Loss: 0.232381, Acc: 0.92, F: 0.49, Fbeta: 0.52, gbeta: 0.31, auroc: 0.89, auprc: 0.62
 
S1/810 6.51 min|
Train Loss: 0.210569, Acc: 0.93, F: 0.55, Fbeta: 0.59, gbeta: 0.38, auroc: 0.91, auprc: 0.70 |
Valid Loss: 0.232114, Acc: 0.92, F: 0.48, Fbeta: 0.51, gbeta: 0.30, auroc: 0.89, auprc: 0.62
 
S1/820 6.60 min|
Train Loss: 0.210114, Acc: 0.94, F: 0.55, Fbeta: 0.58, gbeta: 0.38, auroc: 0.91, auprc: 0.70 |
Valid Loss: 0.231845, Acc: 0.92, F: 0.49, Fbeta: 0.52, gbeta: 0.31, auroc: 0.89, auprc: 0.62
 
S1/830 6.69 min|
Train Loss: 0.209660, Acc: 0.94, F: 0.56, Fbeta: 0.59, gbeta: 0.39, auroc: 0.91, auprc: 0.70 |
Valid Loss: 0.231581, Acc: 0.92, F: 0.48, Fbeta: 0.51, gbeta: 0.31, auroc: 0.89, auprc: 0.62
 
S1/840 6.77 min|
Train Loss: 0.209239, Acc: 0.93, F: 0.54, Fbeta: 0.58, gbeta: 0.37, auroc: 0.91, auprc: 0.70 |
Valid Loss: 0.231350, Acc: 0.92, F: 0.49, Fbeta: 0.52, gbeta

S1/1200 9.92 min|
Train Loss: 0.197490, Acc: 0.94, F: 0.57, Fbeta: 0.61, gbeta: 0.40, auroc: 0.92, auprc: 0.72 |
Valid Loss: 0.225593, Acc: 0.92, F: 0.50, Fbeta: 0.53, gbeta: 0.32, auroc: 0.89, auprc: 0.62
 
S1/1210 10.01 min|
Train Loss: 0.197239, Acc: 0.94, F: 0.54, Fbeta: 0.58, gbeta: 0.37, auroc: 0.92, auprc: 0.71 |
Valid Loss: 0.225502, Acc: 0.92, F: 0.50, Fbeta: 0.53, gbeta: 0.31, auroc: 0.88, auprc: 0.61
 
S1/1220 10.09 min|
Train Loss: 0.197016, Acc: 0.93, F: 0.58, Fbeta: 0.60, gbeta: 0.39, auroc: 0.92, auprc: 0.71 |
Valid Loss: 0.225423, Acc: 0.92, F: 0.47, Fbeta: 0.51, gbeta: 0.29, auroc: 0.88, auprc: 0.61
 
S1/1230 10.18 min|
Train Loss: 0.196818, Acc: 0.94, F: 0.56, Fbeta: 0.60, gbeta: 0.39, auroc: 0.92, auprc: 0.71 |
Valid Loss: 0.225358, Acc: 0.92, F: 0.49, Fbeta: 0.52, gbeta: 0.31, auroc: 0.89, auprc: 0.62
 
S1/1240 10.27 min|
Train Loss: 0.196585, Acc: 0.94, F: 0.58, Fbeta: 0.62, gbeta: 0.41, auroc: 0.92, auprc: 0.72 |
Valid Loss: 0.225254, Acc: 0.92, F: 0.49, Fbeta: 0.

S1/1600 13.38 min|
Train Loss: 0.189256, Acc: 0.94, F: 0.60, Fbeta: 0.63, gbeta: 0.42, auroc: 0.93, auprc: 0.74 |
Valid Loss: 0.222672, Acc: 0.92, F: 0.51, Fbeta: 0.54, gbeta: 0.33, auroc: 0.89, auprc: 0.62
 
S1/1610 13.47 min|
Train Loss: 0.189076, Acc: 0.94, F: 0.60, Fbeta: 0.63, gbeta: 0.42, auroc: 0.93, auprc: 0.73 |
Valid Loss: 0.222618, Acc: 0.92, F: 0.51, Fbeta: 0.54, gbeta: 0.32, auroc: 0.89, auprc: 0.62
 
S1/1620 13.55 min|
Train Loss: 0.188898, Acc: 0.94, F: 0.59, Fbeta: 0.63, gbeta: 0.42, auroc: 0.93, auprc: 0.74 |
Valid Loss: 0.222564, Acc: 0.92, F: 0.51, Fbeta: 0.54, gbeta: 0.33, auroc: 0.89, auprc: 0.63
 
S1/1630 13.64 min|
Train Loss: 0.188727, Acc: 0.94, F: 0.56, Fbeta: 0.60, gbeta: 0.39, auroc: 0.92, auprc: 0.73 |
Valid Loss: 0.222522, Acc: 0.92, F: 0.52, Fbeta: 0.55, gbeta: 0.32, auroc: 0.88, auprc: 0.62
 
S1/1640 13.73 min|
Train Loss: 0.188568, Acc: 0.94, F: 0.57, Fbeta: 0.61, gbeta: 0.40, auroc: 0.92, auprc: 0.73 |
Valid Loss: 0.222479, Acc: 0.92, F: 0.52, Fbeta: 0

...

S8/120 165.55 min|
Train Loss: 0.333299, Acc: 0.90, F: 0.12, Fbeta: 0.13, gbeta: 0.08, auroc: 0.74, auprc: 0.33 |
Valid Loss: 0.334960, Acc: 0.90, F: 0.13, Fbeta: 0.13, gbeta: 0.08, auroc: 0.74, auprc: 0.32
 
S8/130 165.63 min|
Train Loss: 0.329515, Acc: 0.90, F: 0.12, Fbeta: 0.13, gbeta: 0.08, auroc: 0.75, auprc: 0.33 |
Valid Loss: 0.331781, Acc: 0.90, F: 0.13, Fbeta: 0.14, gbeta: 0.08, auroc: 0.74, auprc: 0.32
 
S8/140 165.72 min|
Train Loss: 0.326184, Acc: 0.90, F: 0.13, Fbeta: 0.14, gbeta: 0.09, auroc: 0.75, auprc: 0.33 |
Valid Loss: 0.328991, Acc: 0.90, F: 0.13, Fbeta: 0.13, gbeta: 0.08, auroc: 0.75, auprc: 0.32
 
S8/150 165.81 min|
Train Loss: 0.323220, Acc: 0.90, F: 0.13, Fbeta: 0.14, gbeta: 0.08, auroc: 0.75, auprc: 0.33 |
Valid Loss: 0.326524, Acc: 0.90, F: 0.13, Fbeta: 0.14, gbeta: 0.09, auroc: 0.75, auprc: 0.33
 
S8/160 165.89 min|
Train Loss: 0.320568, Acc: 0.90, F: 0.14, Fbeta: 0.15, gbeta: 0.09, auroc: 0.75, auprc: 0.33 |
Valid Loss: 0.324336, Acc: 0.90, F: 0.13, Fbeta: 0

S8/520 168.99 min|
Train Loss: 0.284157, Acc: 0.90, F: 0.16, Fbeta: 0.16, gbeta: 0.11, auroc: 0.78, auprc: 0.35 |
Valid Loss: 0.299194, Acc: 0.89, F: 0.14, Fbeta: 0.14, gbeta: 0.09, auroc: 0.76, auprc: 0.33
 
S8/530 169.08 min|
Train Loss: 0.283711, Acc: 0.90, F: 0.15, Fbeta: 0.16, gbeta: 0.10, auroc: 0.78, auprc: 0.36 |
Valid Loss: 0.298956, Acc: 0.90, F: 0.15, Fbeta: 0.15, gbeta: 0.10, auroc: 0.76, auprc: 0.33
 
S8/540 169.17 min|
Train Loss: 0.283237, Acc: 0.90, F: 0.17, Fbeta: 0.18, gbeta: 0.12, auroc: 0.78, auprc: 0.36 |
Valid Loss: 0.298689, Acc: 0.90, F: 0.15, Fbeta: 0.16, gbeta: 0.10, auroc: 0.76, auprc: 0.33
 
S8/550 169.25 min|
Train Loss: 0.282747, Acc: 0.90, F: 0.16, Fbeta: 0.18, gbeta: 0.11, auroc: 0.78, auprc: 0.36 |
Valid Loss: 0.298415, Acc: 0.90, F: 0.16, Fbeta: 0.16, gbeta: 0.10, auroc: 0.76, auprc: 0.33
 
S8/560 169.34 min|
Train Loss: 0.282256, Acc: 0.90, F: 0.18, Fbeta: 0.19, gbeta: 0.12, auroc: 0.78, auprc: 0.36 |
Valid Loss: 0.298143, Acc: 0.90, F: 0.16, Fbeta: 0

S8/920 172.43 min|
Train Loss: 0.267398, Acc: 0.92, F: 0.25, Fbeta: 0.26, gbeta: 0.18, auroc: 0.81, auprc: 0.43 |
Valid Loss: 0.290334, Acc: 0.91, F: 0.22, Fbeta: 0.23, gbeta: 0.15, auroc: 0.79, auprc: 0.40
 
S8/930 172.52 min|
Train Loss: 0.267022, Acc: 0.92, F: 0.25, Fbeta: 0.26, gbeta: 0.18, auroc: 0.81, auprc: 0.43 |
Valid Loss: 0.290168, Acc: 0.91, F: 0.21, Fbeta: 0.22, gbeta: 0.14, auroc: 0.79, auprc: 0.40
 
S8/940 172.60 min|
Train Loss: 0.266651, Acc: 0.92, F: 0.24, Fbeta: 0.26, gbeta: 0.17, auroc: 0.81, auprc: 0.43 |
Valid Loss: 0.289994, Acc: 0.91, F: 0.22, Fbeta: 0.23, gbeta: 0.14, auroc: 0.79, auprc: 0.40
 
S8/950 172.69 min|
Train Loss: 0.266275, Acc: 0.92, F: 0.26, Fbeta: 0.27, gbeta: 0.18, auroc: 0.81, auprc: 0.43 |
Valid Loss: 0.289828, Acc: 0.91, F: 0.21, Fbeta: 0.22, gbeta: 0.14, auroc: 0.79, auprc: 0.40
 
S8/960 172.78 min|
Train Loss: 0.265916, Acc: 0.92, F: 0.24, Fbeta: 0.25, gbeta: 0.17, auroc: 0.82, auprc: 0.43 |
Valid Loss: 0.289677, Acc: 0.90, F: 0.23, Fbeta: 0

S8/1320 175.87 min|
Train Loss: 0.254627, Acc: 0.92, F: 0.31, Fbeta: 0.34, gbeta: 0.21, auroc: 0.83, auprc: 0.47 |
Valid Loss: 0.285593, Acc: 0.91, F: 0.29, Fbeta: 0.31, gbeta: 0.18, auroc: 0.80, auprc: 0.43
 
S8/1330 175.95 min|
Train Loss: 0.254350, Acc: 0.92, F: 0.32, Fbeta: 0.35, gbeta: 0.22, auroc: 0.83, auprc: 0.47 |
Valid Loss: 0.285517, Acc: 0.91, F: 0.30, Fbeta: 0.32, gbeta: 0.18, auroc: 0.80, auprc: 0.43
 
S8/1340 176.04 min|
Train Loss: 0.254079, Acc: 0.92, F: 0.32, Fbeta: 0.35, gbeta: 0.22, auroc: 0.83, auprc: 0.47 |
Valid Loss: 0.285449, Acc: 0.91, F: 0.29, Fbeta: 0.31, gbeta: 0.18, auroc: 0.80, auprc: 0.43
 
S8/1350 176.13 min|
Train Loss: 0.253809, Acc: 0.92, F: 0.34, Fbeta: 0.36, gbeta: 0.23, auroc: 0.83, auprc: 0.48 |
Valid Loss: 0.285379, Acc: 0.91, F: 0.28, Fbeta: 0.30, gbeta: 0.17, auroc: 0.80, auprc: 0.42
 
S8/1360 176.21 min|
Train Loss: 0.253557, Acc: 0.92, F: 0.32, Fbeta: 0.34, gbeta: 0.21, auroc: 0.83, auprc: 0.47 |
Valid Loss: 0.285331, Acc: 0.91, F: 0.28, Fbe

S8/1720 179.30 min|
Train Loss: 0.245637, Acc: 0.93, F: 0.36, Fbeta: 0.39, gbeta: 0.25, auroc: 0.83, auprc: 0.49 |
Valid Loss: 0.283977, Acc: 0.92, F: 0.32, Fbeta: 0.35, gbeta: 0.20, auroc: 0.80, auprc: 0.44
 
S8/1730 179.39 min|
Train Loss: 0.245438, Acc: 0.93, F: 0.37, Fbeta: 0.40, gbeta: 0.26, auroc: 0.83, auprc: 0.49 |
Valid Loss: 0.283968, Acc: 0.91, F: 0.31, Fbeta: 0.33, gbeta: 0.19, auroc: 0.80, auprc: 0.43
 
S8/1740 179.48 min|
Train Loss: 0.245246, Acc: 0.93, F: 0.35, Fbeta: 0.39, gbeta: 0.24, auroc: 0.83, auprc: 0.49 |
Valid Loss: 0.283963, Acc: 0.91, F: 0.33, Fbeta: 0.35, gbeta: 0.20, auroc: 0.80, auprc: 0.44
 
S8/1750 179.56 min|
Train Loss: 0.245053, Acc: 0.93, F: 0.36, Fbeta: 0.39, gbeta: 0.25, auroc: 0.83, auprc: 0.49 |
Valid Loss: 0.283956, Acc: 0.91, F: 0.33, Fbeta: 0.35, gbeta: 0.20, auroc: 0.80, auprc: 0.44
 
